In [218]:
import random
import jieba.analyse
import pandas as pd

## 1. 加载停用词

In [219]:
# stop_words_path = "C:\\Users\\Name\\PycharmProjects\\NLP_project\\resources\\stopwords-master\\百度停用词表.txt"
# stop_words_path = "C:\\Users\\Name\\PycharmProjects\\NLP_project\\resources\\stopwords-master\\哈工大停用词表.txt"
stop_words_path = "/home/yang/PycharmProjects/NLP_projects/resources/stopwords-master/哈工大停用词表.txt"
# stop_words_path = "/home/yang/PycharmProjects/NLP_projects/resources/stopwords-master/中文停用词表.txt"
# stop_words_path = "/home/yang/PycharmProjects/NLP_projects/resources/stopwords-master/modified_stopwords.txt"

# stop_words_path = "/home/yang/PycharmProjects/NLP_project/resources/stopwords-master/中文停用词表.txt"
# stop_words_path = "/home/yang/PycharmProjects/NLP_project/resources/stopwords-master/百度停用词表.txt"
# stop_words_path = "/home/yang/PycharmProjects/NLP_project/resources/stopwords-master/modified_stopwords.txt"
# stop_words_path = "/home/yang/PycharmProjects/NLP_project/resources/stopwords-master/四川大学机器智能实验室停用词库.txt"

stop_words = pd.read_csv(stop_words_path,index_col=False,quoting=3,sep="\t",names=['stopword'], encoding='utf-8')
stop_words = stop_words['stopword'].values

## 2. 加载语料

In [220]:
# data_path = "C:\\Users\\Name\\PycharmProjects\\NLP_project\\resources\\面试-训练测试集.xlsx"
data_path = "/home/yang/PycharmProjects/NLP_projects/resources/面试-训练测试集.xlsx"
predict_data_path = "/home/yang/PycharmProjects/NLP_projects/resources/面试-待预测集.xlsx"
# data_path = "/home/yang/PycharmProjects/NLP_project/resources/面试-训练测试集.xlsx"
data = pd.read_excel(data_path)
predict_data = pd.read_excel(predict_data_path)


## 3.定义文本预处理函数

* 去无关字符，去停用词，分词
* 参数 titles, contents, sentences, labels

In [221]:
def is_float(s):
    s = str(s)
    if s.count('.') ==1:
        left = s.split('.')[0]
        right = s.split('.')[1]
        if right.isdigit():
            if left.count('-')==1 and left.startswith('-'):
                num = left.split['-'][-1]
                if num.isdigit():
                    return True
            elif left.isdigit():
                return True
    return False

def is_percents(s):
    s = str(s)
    if s.count('.') ==1:
        left = s.split('.')[0]
        right = s.split('.')[1]
        right = right.split('%')[0]
        if right.isdigit():
            if left.count('-')==1 and left.startswith('-'):
                num = left.split['-'][-1]
                if num.isdigit():
                    return True
            elif left.isdigit():
                return True
            
    elif s.count(".") != 1:
        left = s.split('%')[0]
        if left.isdigit():
            return True
               
    return False

def is_all_chinese(strs):
    for _char in strs:
        if not '\u4e00' <= _char <= '\u9fa5':
            return False
    return True


def preprocess_text(title, content, sentences, label):
    
#     处理标题
    title_info = jieba.lcut(title)

    title_info = [v for v in title_info if not str(v).isdigit()]#去数字
    title_info = list(filter(lambda x:x.strip(), title_info))   #去左右空格
    

    title_info = list(filter(lambda x:len(x)>1, title_info)) #长度为1的字符
    title_info = list(filter(lambda x:x not in stop_words, title_info)) #去掉停用词
            
#     处理文章内容        
    segs=jieba.lcut(content)
    segs = [v for v in segs if not str(v).isdigit()]#去数字
    segs = [v for v in segs if not is_float(v)] #去小数
    segs = [v for v in segs if not is_percents(v)] #去百分数

    segs = list(filter(lambda x:x.strip(), segs))   #去左右空格
    segs = list(filter(lambda x:len(x)>1, segs)) #长度为1的字符
    segs = list(filter(lambda x:x not in stop_words, segs)) #去掉停用词
    
    
#     print("内容信息是:", segs)
    segs = title_info + segs  
    segs = [v for v in segs if is_all_chinese(str(v))] #去字母词
    
    sentences.append((" ".join(segs), label))# 打标签

        
    


## 4. 文本预处理

In [222]:
sentences = []

for index, row in data.iterrows():
#     print(row['title'], row['content'])
    title = row['title']
    content = row['content']
    label = row['label']
    preprocess_text(title, content, sentences, label)
    

In [223]:
# random.shuffle(sentences)
# for sentence in sentences[:10]:
#     print("-------")
#     print("文章分词:", sentence[0])
#     print("文章标签：", sentence[1])

## 5. 抽取词向量特征

* 测试了两种提取方法，CountVectorizer词频提取和TfidfVectorizer的td-idf特性提取
* CountVectorizer提取词频特征后，在训练集上有明显过拟合现象
* Tf-idf模型能更有效避免噪声词的干扰，在测试集上表现更好，这里用Tf-idf来构建词袋模型

In [224]:
# 1. 抽取特征， 定义词袋模型

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# vec = CountVectorizer(
#     analyzer='word', # tokenise by character ngrams
#     ngram_range=(1,2),  # use ngrams of size 1 and 2
#     max_features=1000,  # keep the most common 1000 ngrams
# )

vec = TfidfVectorizer(
    analyzer='word', # tokenise by character ngrams
    ngram_range=(1,3),  # use ngrams of size 1 and 2
    max_features= None,  # keep the most common 1000 ngrams
    use_idf=1,smooth_idf=1,sublinear_tf=1
)



In [225]:
# 2. 语料切分, 随机分出测试集用于验证，比例70%训练集，30%测试集

from sklearn.model_selection import train_test_split
x, y = zip(*sentences)
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.3, random_state=1256)
# print(x_train)


## 6. 定义模型

In [226]:
# 3. 把训练数据转为词袋模型
vec.fit(x_train)
x_train_data = vec.transform(x_train)

In [227]:
# 4.算法建模与模型训练 （贝叶斯）
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(x_train_data, y_train)
# 5. 评估结果 AUC值
print("多项贝叶斯模型评价，训练集正确率:",classifier.score(vec.transform(x_train), y_train))


多项贝叶斯模型评价，训练集正确率: 0.9054441260744985


In [228]:
# 4.算法建模与模型训练 （SVM）

from sklearn.svm import SVC
# svm = SVC(kernel='linear')
svm = SVC(C=1.0,kernel='linear',gamma='auto')
svm.fit(x_train_data, y_train)

print("SVM模型评价，训练集正确率",svm.score(x_train_data, y_train))


SVM模型评价，训练集正确率 0.9684813753581661


In [229]:
import sklearn.neural_network as sk_nn
model = sk_nn.MLPClassifier(activation='tanh',solver='sgd',alpha=0.0001,learning_rate='adaptive',learning_rate_init=0.01,max_iter=2000)
model.fit(x_train_data,y_train)

print("神经网络模型评价，训练集正确率", model.score(x_train_data, y_train))

acc=model.score(vec.transform(x_test),y_test) #根据给定数据与标签返回正确率的均值


神经网络模型评价，训练集正确率 0.9856733524355301


## 7.检验模型

In [230]:
print("多项贝叶斯模型评价，测试集正确率:",classifier.score(vec.transform(x_test), y_test))
print("SVM模型评价，测试集正确率",svm.score(vec.transform(x_test), y_test))
print('神经网络模型评价, 测试集正确率:',acc)

多项贝叶斯模型评价，测试集正确率: 0.8533333333333334
SVM模型评价，测试集正确率 0.9133333333333333
神经网络模型评价, 测试集正确率: 0.9133333333333333


### 目前准确率最高的模型是神经网络，在测试集上为92%左右， SVM模型在测试集上准确率91%左右，但训练速度更快。这里我们保存神经网络的模型

In [231]:
from sklearn.externals import joblib

# 保存模型
joblib.dump(model,'mlp.model') 
# 加载模型
mlp_model=joblib.load('mlp.model')



## 8. 预测分类

In [232]:
# 处理预测数据
predict_data_sentences = []

for index, row in data.iterrows():
#     print(row['title'], row['content'])
    title = row['title']
    content = row['content']
    preprocess_text(title, content, predict_data_sentences, label= None)

x, none_labels = zip(*predict_data_sentences)
# vec.fit(x)
x = vec.transform(x)

# print(x)
# 应用模型进行预测
result=mlp_model.predict(x)

print(result)

['正面' '正面' '负面' '负面' '负面' '负面' '负面' '正面' '负面' '负面' '负面' '负面' '正面' '正面'
 '正面' '正面' '正面' '负面' '正面' '负面' '正面' '正面' '正面' '正面' '正面' '正面' '正面' '正面'
 '正面' '中性' '正面' '正面' '负面' '正面' '正面' '正面' '正面' '负面' '负面' '负面' '正面' '负面'
 '正面' '正面' '正面' '正面' '正面' '正面' '正面' '正面' '正面' '负面' '正面' '负面' '正面' '正面'
 '负面' '正面' '正面' '正面' '正面' '正面' '正面' '正面' '正面' '正面' '负面' '正面' '正面' '正面'
 '正面' '正面' '正面' '负面' '正面' '正面' '正面' '负面' '正面' '正面' '正面' '正面' '正面' '正面'
 '负面' '正面' '正面' '正面' '正面' '正面' '正面' '正面' '负面' '正面' '正面' '正面' '负面' '正面'
 '正面' '正面' '正面' '正面' '正面' '正面' '正面' '正面' '正面' '负面' '正面' '正面' '负面' '正面'
 '正面' '正面' '正面' '中性' '正面' '负面' '负面' '正面' '中性' '正面' '正面' '正面' '负面' '正面'
 '正面' '正面' '正面' '正面' '正面' '正面' '正面' '正面' '正面' '正面' '中性' '正面' '负面' '正面'
 '正面' '正面' '正面' '正面' '正面' '正面' '负面' '正面' '正面' '正面' '正面' '正面' '正面' '负面'
 '正面' '负面' '正面' '正面' '正面' '负面' '正面' '正面' '正面' '正面' '正面' '正面' '正面' '负面'
 '正面' '负面' '正面' '负面' '负面' '正面' '中性' '正面' '正面' '正面' '正面' '正面' '正面' '负面'
 '负面' '正面' '正面' '正面' '正面' '正面' '正面' '正面' '正面' '正面' '正面' '负面' '正面' '正面'
 '正面' 

In [249]:
from pyecharts.charts import Pie

# bar = Bar()
# bar.add_xaxis(["衬衫", "羊毛衫", "雪纺衫", "裤子", "高跟鞋", "袜子"])
# bar.add_yaxis("商家A", [5, 20, 36, 10, 75, 90])
# bar.add_yaxis("商家B", [6, 20, 26, 10, 65, 100])
# # render 会生成本地 HTML 文件，默认会在当前目录生成 render.html 文件
# # 也可以传入路径参数，如 bar.render("mycharts.html")
# bar.render_notebook()


pos = 0
neg = 0
neu = 0

for i in result:
    if i == '正面':
        pos += 1
    elif i == '负面':
        neg += 1
    elif i == '中性':
        neu += 1
        
res = [pos, neg, neu] 
print(res)
pie = Pie("类别分析",**style.init_style)

attr = ['正面','负面','中性']

pie.add("",attr, res)

# pie.add('类别信息', [pos, neg, neu], is_label_show=True, legend_pos='left', label_text_color=None, legend_orient='vertical', radius=[30, 75])

#     pie.render('Pie-weather.html')
pie.render_notebook()


ImportError: cannot import name 'style' from 'pyecharts.charts' (/home/yang/anaconda3/lib/python3.7/site-packages/pyecharts/charts/__init__.py)